In [ ]:
# Install the necessary libraries
!pip install gradio transformers torch nltk
!pip install gradio
!pip install transformers
!pip install torch
!pip install nltk

In [ ]:
# Import necessary libraries
import gradio as gr
import torch
from transformers import pipeline
from nltk.tokenize import word_tokenize

# Download NLTK data
import nltk
nltk.download('punkt')

# Initialize Hugging Face transformers for sentiment analysis and bias detection
sentiment_pipeline = pipeline("sentiment-analysis")
gender_bias_model = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Gender-specific words to detect bias
masculine_words = ["man","men","sir","gentleman","aggressive", "competitive", "dominant", "decisive", "leader", "strong", "confident", "assertive"]
feminine_words = ["female","lady","madam","cooperative", "nurturing", "supportive", "compassionate", "gentle", "caring", "empathic", "friendly"]

# Function to preprocess the text (tokenize)
def preprocess_text(text):
    return word_tokenize(text.lower())

# Function to detect gendered language and sentiment in a job description
def detect_bias_and_sentiment(job_description):
    # Tokenize and count gendered language
    words = preprocess_text(job_description)
    masculine_count = sum([word in masculine_words for word in words])
    feminine_count = sum([word in feminine_words for word in words])

    # Detect gender bias based on word counts
    if masculine_count > feminine_count:
        gender_bias_feedback = "The job description contains more masculine-coded language."
    elif feminine_count > masculine_count:
        gender_bias_feedback = "The job description contains more feminine-coded language."
    else:
        gender_bias_feedback = "The job description appears neutral in gender-coded language."

    # Perform sentiment analysis
    sentiment_result = sentiment_pipeline(job_description)
    sentiment_feedback = f"Sentiment: {sentiment_result[0]['label']} with a score of {sentiment_result[0]['score']:.2f}"

    # Perform zero-shot classification to detect gender bias (can be extended for other types of bias)
    zero_shot_result = gender_bias_model(job_description, candidate_labels=["male", "female", "neutral"])
    bias_label = zero_shot_result['labels'][0]
    bias_score = zero_shot_result['scores'][0]

    # Generate bias feedback and score
    bias_feedback = f"Bias detection (zero-shot classification): {bias_label.capitalize()} bias with a score of {bias_score:.2f}"

    return gender_bias_feedback, sentiment_feedback, bias_feedback

# Create Gradio interface
def gradio_interface(job_description):
    gender_bias_feedback, sentiment_feedback, bias_feedback = detect_bias_and_sentiment(job_description)
    return gender_bias_feedback, sentiment_feedback, bias_feedback

# Set the Gradio interface with labels for outputs
iface = gr.Interface(
    fn=gradio_interface,
    inputs="text",  # Input type is text box
    outputs=[
        gr.Textbox(label="Gender Bias Feedback"),  # First output: Gender Bias Feedback
        gr.Textbox(label="Sentiment Feedback"),  # Second output: Sentiment Feedback
        gr.Textbox(label="Bias Detection Feedback")  # Third output: Bias Detection Feedback
    ],
    title="Job Description Gender Bias and Sentiment Analysis",
    description="This tool analyzes job descriptions for gender bias and provides feedback on the sentiment.",
    theme="compact"  # You can try other themes like "huggingface", or "default"
)

# Customize the appearance with custom CSS (using a lilac and purple color theme)
iface.css = """
    .gradio-container {
        background-color: #F3E5F5;  /* Lilac background */
    }
    .gradio-title {
        color: #6A1B9A;  /* Purple title font */
        font-family: 'Arial', sans-serif;
    }
    .gradio-description {
        color: #6A1B9A;  /* Purple description font */
    }
    .gradio-input textarea {
        background-color: #F8E1F4;  /* Light lilac for input box */
        color: #6A1B9A;  /* Purple font for input text */
    }
    .gradio-output textarea {
        background-color: #F8E1F4;  /* Light lilac for output box */
        color: #6A1B9A;  /* Purple font for output text */
    }
"""

# Launch the Gradio interface
iface.launch(share=True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:1020: UserWarning: Cannot load compact. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/compact (Request ID: Root=1-67323b61-2a2e7a633298822118e1c80b;3af31493-3378-4a33-b61f-082c086862d0)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://605008a8efc0c1f97f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
